In [1]:
import casadi as ca

# load data files
# PLEASE MODIFY THE PATH OF DATA!!!!!!!!!!!!!!!!!!!!!!!!!
# ===========================================================================================
# VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
# # select data
data_subfold = "data_2023_03_01-16_48_29"    # the data from Lukas
# data_subfold = "rosbag2_2023_06_05-16_57_44"
# data_subfold = "rosbag2_2023_06_05-17_22_10"
# data_subfold = "rosbag2_2023_06_05-17_26_41"

# data_subfold = "rosbag2_2023_06_16-16_50_38"    # data for correction of imu coordinate 

# merge_command = False
save_merged_data = False

execfile("data_process_from_rosbags.py")
# AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
# ===========================================================================================

'data.hamster2__command.csv' loaded to 'df_command'
'data.hamster2__imu.csv' loaded to 'df_imu'
'data.hamster2__velocity.csv' loaded to 'df_velocity'
'data.tf.csv' loaded to 'df_tf'

Defined Variables:
    df_command
    df_imu
    df_velocity
    df_tf
    df_merge    # merged from 'df_command', 'df_velocity', 'df_imu' and 'df_tf'

Finish!


In [2]:
Dimu = df_imu.iloc[:,[1, -1, 3]].rename(columns={'imu.header.stamp': 'time'})

Dvelocity = df_velocity.rename(columns={'__time': 'time'}) # global time

Dtf = df_tf.iloc[:,[1, 2, 3, -1]].rename(columns={'tf.header.stamp': 'time'})# use header.stamp

# time difference between header.stamp and __time of TF in the first data point
d_time = df_tf.iloc[0,1] - df_tf.iloc[0,0]


Dtf["time"] = Dtf["time"] - d_time


df_data = Dimu.merge(Dvelocity, how='outer', on='time')
df_data = df_data.merge(Dtf, how='outer', on='time')
df_data.sort_values(by=['time'], inplace=True)
df_data.reset_index(inplace=True, drop=True)
data = df_data.to_numpy()
df_data.head(10)


,time,imu.a_z,imu.w_y,velocity.v,tf.x,tf.y,tf.yaw_z
0,1.677686e+09,0.519752,0.001745,NaN,NaN,NaN,NaN
1,1.677686e+09,0.519752,0.000000,NaN,NaN,NaN,NaN
2,1.677686e+09,0.519752,0.000000,NaN,NaN,NaN,NaN
3,1.677686e+09,0.519752,0.001745,NaN,NaN,NaN,NaN
4,1.677686e+09,0.519752,0.003491,NaN,NaN,NaN,NaN
5,1.677686e+09,0.519752,0.001745,NaN,NaN,NaN,NaN
6,1.677686e+09,NaN,NaN,NaN,2.668901,-3.346754,-0.076459
7,1.677686e+09,NaN,NaN,0.0,NaN,NaN,NaN
8,1.677686e+09,NaN,NaN,NaN,2.668912,-3.346767,-0.076442
9,1.677686e+09,0.519752,0.000000,NaN,NaN,NaN,NaN


In [ ]:
df_data.iloc[0,3]==np.nan

In [ ]:
def ode(x, u):
    """
    ODE equations
        
     dx = [x[3] * ca.cos(x[2]),
           x[3] * ca.sin(x[2]),
           -u[1],
           u[0]])
    
    :param x: [x, y, theta, v].T  , all system states
    :param u: [a_z, w_y].T  , 'inputs' of the system
    :return: dot_x
    """
    A = np.array([[0, 0, 0, np.cos(x[2])],
                  [0, 0, 0, np.sin(x[2])],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0]])
    B = np.array([[0, 0],
                  [0, 0],
                  [0, -1],
                  [1, 0]])
    dx = A @ x + B @ u

    return dx

def f_next_x(x, u, Ts):
    """
    RK-4 integral

    :param x:
    :param u:
    :param Ts:
    :return:
    """
    k1 = ode(x, u)
    k2 = ode(x + Ts / 2 * k1, u)
    k3 = ode(x + Ts / 2 * k2, u)
    k4 = ode(x + Ts * k3, u)

    x_next = x + 1 / 6 * Ts * (k1 + 2 * k2 + 2 * k3 + k4)
    return x_next

def f_next_x_plus_w(x, u, Ts, w):
    """
    RK-4 integral

    :param x:
    :param u:
    :param Ts:
    :return:
    """
    x_next = f_next_x(x, u, Ts) + w
    return x_next

class MHE_hamster():
    def __init__(self, horizon, Q, R_v, R_t):
        
        self.horizon = horizon        # N
        
        # # Info holder
        self.x_holder = np.zeros(4, horizon + 1)
        self.y_holder = None          # list[tuple(str, numpy.ndarray)]
        self.u_holder = np.zeros(2, horizon)
        
        # # Ojective Function
        self.x0_sym = ca.SX.sym('x0', 4, 1)
        self.x_sym_seq = ca.SX.sym('x_sym_seq', 4, horizon + 1)    # x_0 ~ x_N,   X = [x0, x1, ..., xN], shape:(4, N+1)
        self.w_sym_seq = ca.SX.sym('w_sym_seq', 4, horizon)        # w_0 ~ w_N-1
        self.u_sym_seq = ca.SX.sym('u_sym_seq', 2, horizon)        # u_0 ~ u_N-1

        # states noise ||w||
        sum_w_matrix = np.multiply(w_sym_seq, Q @ w_sym_seq)            # np.multiply() element wise
        sum_w = ca.SX.ones(1,4) @ sum_w_matrix @ ca.SX.ones(horizon,1)  # same as sum 'for' loop and faster

        # state sequence X = [x0, x1, ..., xN], shape:(4, N+1)
        self.x_sym_seq[:,0] = self.x0_sym
        for i in range(horizon):
            self.x_sym_seq[:, i + 1] = f_next_x_plus_w(self.x_sym_seq[:, i], self.u_sym_seq[:, i], Ts, self.w_sym_seq[:, i])
        
    def update_obj():
        pass
    def forward(self, y_mea: tuple, u: np.ndarray):
        if self.y_holder is None:
            self.y_holder = [y_mea] * self.horizon
 
        
        
        
        
        
        

In [ ]:
s_imu = np.isnan(data[:, 1])
s_vel = np.isnan(data[:, 3])
s_tf = np.isnan(data[:, 4])
s_m = s_imu * s_vel
s = s_m * s_tf

print(any(s_m))
print(any(s))

In [ ]:
from casadi import *

# Declare variables
x = SX.sym("x", 2)
p = SX.sym("p", 2)

# Form the NLP
f = (x[0] - p[0]) ** 2 + (x[1] - p[1]) ** 2  # objective
g = x[0] + x[1] -6  # constraint

# nlp = {'x': x, 'f': f, 'g': g}
nlp = {'x': x, 'f': f, 'g': g, 'p': p} # p: parameter


# Pick an NLP solver
MySolver = "ipopt"
# MySolver = "worhp"
# MySolver = "sqpmethod"

# Solver options
opts = {}
if MySolver == "sqpmethod":
    opts["qpsol"] = "qpoases"
    opts["qpsol_options"] = {"printLevel": "none"}

# Allocate a solver
solver = nlpsol("solver", MySolver, nlp, opts)

# Solve the NLP
x0 = DM([0,0])
p0 = DM([2,3])
sol = solver(x0=x0, p=p0, lbg=0)

# Print solution
print("-----")
print("objective at solution = ", sol["f"])
print("primal solution = ", sol["x"])
print("dual solution (x) = ", sol["lam_x"])
print("dual solution (g) = ", sol["lam_g"])

In [ ]:
x = ca.SX.sym("x", 2,1)
a = np.array([[1],[2],[2]])
a[:-1] = np.array([[3],[3]])
np.append(np.zeros(1),2)

In [ ]:
df_data.head(10)

In [3]:
dict_init = {'time': [0.], 
             'sensor.type': ['imu'], 
             'U.a_z': [0.], 
             'U.w_y': [0.], 
             'x': [0.], 
             'y': [0.], 
             'theta': [0.], 
             'v': [0.]}
dict_0 = {'time': [0.1], 
             'sensor.type': ['tf'], 
             'tf.x': [0.], 
             'tf.y': [0.], 
             'tf.yaw_z': [0.]}
dict_1 = {'time': [0.2], 
             'sensor.type': ['vel'], 
             'velocity': [0.]}
dict_00 = {'time': [0.25], 
             'sensor.type': ['imu'], 
             'U.a_z': [0.], 
             'U.w_y': [0.], 
             'x': [0.], 
             'y': [0.], 
             'theta': [0.], 
             'v': [0.]}
dict_2 = {'time': [0.3], 
             'sensor.type': ['tf'], 
             'tf.x': [0.], 
             'tf.y': [0.], 
             'tf.yaw_z': [0.]}
dict_3 = {'time': [0.4], 
             'sensor.type': ['vel'], 
             'velocity': [0.]}

df_all = pd.DataFrame(dict_init)
df_0 = pd.DataFrame(dict_0)
df_1 = pd.DataFrame(dict_1)
df_00 = pd.DataFrame(dict_00)
df_2 = pd.DataFrame(dict_2)
df_3 = pd.DataFrame(dict_3)


df_all = pd.concat((df_all,df_0, df_1, df_00, df_2, df_3), axis=0, ignore_index=True)
df_tep = df_all.sort_values(by=['time'], inplace=False, ascending=True)

df_all.head(10)


,time,sensor.type,U.a_z,U.w_y,x,y,theta,v,tf.x,tf.y,tf.yaw_z,velocity
0,0.00,imu,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0.10,tf,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
2,0.20,vel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0.25,imu,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,0.30,tf,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
5,0.40,vel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [5]:
df_mea = df_all[df_all['sensor.type']!='imu']
df_mea.head(10)

,time,sensor.type,U.a_z,U.w_y,x,y,theta,v,tf.x,tf.y,tf.yaw_z,velocity
1,0.1,tf,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
2,0.2,vel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0.3,tf,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
5,0.4,vel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [7]:
df_mea[['x', 'y', 'theta', 'v','tf.x']].iloc[:-1].to_numpy()

array([[nan, nan, nan, nan,  0.],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan,  0.]])

In [4]:
list_id = df_mea.iloc[-3:,:].index.to_list()
list_id[0]
print(list_id)
df_all_horizon = df_all.loc[list_id[0]:]
df_all_horizon

NameError: name 'df_mea' is not defined

In [ ]:
a = ca.SX.sym("a", 2,1)
b = ca.SX.sym("b", 2,1)
x = ca.SX.sym("x", 2,1)
u = ca.SX.sym("u", 2,1)

f = ca.Function('f',[a,b],[a+b])
f(np.array([[2],[2]]),u)